In [163]:
import numpy as np
import pandas as pd
from scipy import special
import scipy
import scipy.linalg
import types
import math
from itertools import combinations
from timeit import default_timer as timer 
import csv
import os
import sys
import matplotlib.pyplot as plt
from matplotlib.legend import Legend
from matplotlib.pyplot import figure
#Make sure to change your directory 
#os.mkdir("plots")

#### Use the code below with the Main Code 

## Cleaning Raw Data
#### The code below is from the Spectral Analysis (Main Code). It reduces raw data by collapsing 26 gene columns to 13 

In [164]:
#Import raw data 
gene_dat=pd.read_csv('vcf_FA.csv',index_col='Unnamed: 0',encoding='utf-8')
matrix=gene_dat.as_matrix()

/Users/msri/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [165]:
#this code is from the Main Code. Reduces raw data by collapsing 26 columns to 13
gene_mut=np.zeros((matrix.shape[0],int(np.ceil((matrix.shape[1]))/2)))
                                   
for rows in range(matrix.shape[0]):
    for cols in range(0,matrix.shape[1]-1,2): #excluding phenotype column
        if(matrix[rows][cols]==1 or matrix[rows][cols+1]==1):  #"or" can be changed to "and"
            gene_mut[rows][int(cols/2)]=1 
            
#adding back the phenotype column       
gene_mut[:,-1]=matrix[:,-1]


#### This code is new. Not in Spectral Analysis (Main Code)

In [1]:
# getting column names of genes 
# names=gene_dat.columns
# new_colnames=[]
# for n in range(0,len(names)-1,2):
#     new_colnames.append(names[n].split("-")[0])
# gene_names=list(new_colnames)
# print(gene_names)

#change gene names to alphabet names (user friendly)
gene_names = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M"]
print(gene_names) 

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M']


## Helper Functions (for Color Peaks Code)

In [2]:
#this function inputs the 13 gene names and the kth grouping of f...
#...and outputs a list of all k combinations of gene names in lexicalgraphical order
def create_list_names(gene_names,k):
    list_names=[]
    for i in range(1,k+1):
        list_names.append(alphabet_choose_k(gene_names,i))
    return list_names

#this function inputs the list of all k combinations of gene names in lexicographical order
#... and outputs the indices (key) for each gene group-name in list_names
def dict_names(list_names):
    keys=range(1,len(list_names)+1)
    return (dict(zip(keys,list_names)))   

# this function inputs a Mallow list with specified order and kth grouping...
#...and outputs a single f_i mallow vector from the kth group
def Mallow_f_order(MallowList, order, k):
    for tup in MallowList:
        if (int(tup[0]))==int(k):
            list_fi= tup[1]
            return list_fi[order]
    
# input: a single mallow vector
# output: returns a list of indices which contain max vals
def get_n_max(f_i, n_max_peak):
    n = n_max_peak
    indices = f_i.ravel().argsort()[-n:]  
    top_ind=[i for i in indices]
    top_ind.reverse()
    return top_ind

#similar function as above. Returns list of indices with min vals
def get_n_min(f_i, n_min_peak):
    n= n_min_peak
    indices = f_i.ravel().argsort()[:n]
    bot_ind=[i for i in indices]
    return bot_ind

#this function retrieves the peak name of max peaks, min peaks 
def retrieve_peak_names(diction,top_ind,order):
    order_gene_names=[]
    for i in top_ind:
        order_gene_names.append(diction.get(order)[i])
    return order_gene_names


## Main Color-Peaks Code

In [172]:
#this function inputs a Mallow list, the i'th order, the kth partition, and the num of peaks...
#... and outputs a bar-plot of the i'th order effects within gene groupings of k...
#...with the max and min peaks colored
def color_peaks(MallowList, order, k, n_max_peak, n_min_peak):
    #all inputs from helper functions 
    f_i = Mallow_f_order(MallowList, order, k)
    top_ind=get_n_max(f_i, n_max_peak)
    bot_ind=get_n_min(f_i,n_min_peak)
    list_names= create_list_names(gene_names,k)
    diction=dict_names(list_names)
    top_peak_gene_names=retrieve_peak_names(diction, top_ind, order)
    top_min_peak_gene_names=retrieve_peak_names(diction, bot_ind, order)
   
    #the codes below changes the size and location of things ###############################
    x_axis_1 = np.arange(len(f_i))
    #width changes size of bars
    plt.bar(x_axis_1,f_i, width= 0.8, align='center', alpha=0.5)
    #change graph size
    #plt.rcParams['figure.figsize']=[10,6]
    
    #the actual plotting##########################
    #x label
    plt.xlabel('Lexicographical order of gene combinations')
    plt.ylim((min(f_i)+10),(max(f_i)-10))
    for i in range(len(top_ind)):
        plt.bar(top_ind[i],f_i[top_ind[i]], align='center',label=top_peak_gene_names[i])
    for j in range(len(bot_ind)):
        plt.bar(bot_ind[j],f_i[bot_ind[j]], align='center',label=top_min_peak_gene_names[j])
    #change location of legend
    plt.legend(bbox_to_anchor=(1.3, 1))
    
    #naming the plots#########################
    if(int(order) == 1):
        plt.title('%dst order effects within groupings of %d mutations'%(order, k))
    elif(int(order) == 2):
        plt.title('%dnd order effects within groupings of %d mutations'%(order, k))
    elif(int(order) == 3):
        plt.title('%drd order effects within groupings of %d mutations'%(order, k))
    else:
        plt.title('%dth order effects within groupings of %d mutations'%(order, k))
        
    #save plots #############################
    #plt.savefig("peak_plots/%d_Order_13_c_%d.png"%(order, k), bbox_inches="tight")
    #plt.show()
    

In [173]:
#k= 5 or 6 mutations 
#order range from 1 to 6
#color_peaks(AllMallowVectors2, order=1, k=5, n_max_peak=3, n_min_peak=4) 


In [170]:
#k= 7,8,9, or 10 mutations 
#order range from 1 to 4
#color_peaks(AllMallowVectors, order=6, k=6, n_max_peak=5, n_min_peak=4)
  